# Intro

## Packages

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

## Data-frame 

In [2]:
df = pd.read_csv("features.csv", sep=";")
df

df["Label"].value_counts()

Label
objective     635
subjective    365
Name: count, dtype: int64

## Nettoyage de la base 

In [3]:
df = df.drop('semanticobjscore', axis=1) 
df = df.drop('semanticsubjscore', axis=1) 
df = df.drop('TextID', axis=1) 
df = df.drop('URL', axis=1) 

df

,Label,totalWordsCount,CC,CD,DT,EX,FW,INs,JJ,JJR,...,pronouns2nd,pronouns3rd,compsupadjadv,past,imperative,present3rd,present1st2nd,sentence1st,sentencelast,txtcomplexity
0,objective,109,7,9,0,5,8,6,0,0,...,0,3,0,11,0,0,0,0,1,18
1,objective,309,1,19,1,4,35,23,0,0,...,0,10,0,13,0,14,9,1,1,14
2,objective,149,8,14,0,5,15,11,0,0,...,0,2,0,8,0,3,2,1,1,18
3,objective,305,7,26,0,10,37,21,1,1,...,0,8,3,13,1,7,1,1,1,20
4,objective,491,33,47,0,12,61,36,0,1,...,0,16,2,34,1,5,6,1,1,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,subjective,926,5,83,1,20,96,63,1,4,...,16,46,10,13,12,49,29,1,1,24
996,subjective,876,9,109,1,16,89,49,2,1,...,8,43,5,34,9,19,31,1,1,21
997,subjective,1469,14,171,1,10,157,106,5,6,...,9,49,12,40,19,51,42,1,1,18
998,subjective,343,4,24,0,9,28,20,0,0,...,0,8,1,3,3,25,7,1,1,11


On supprime les scores d'objectivité et de subjectivité qui informent trop notre modèle et rendent presque caduc la suite du projet. 

In [4]:
#X = df.drop('Label', axis=1) 
#y = df['Label'] 

# Séparation train / test (80% / 20%)
#X_train, X_test, y_train, y_test = train_test_split(
#    X, y,
#    test_size=0.2,     
#    random_state=42,   
 #   shuffle=True,       
  #  stratify= y       # Pour pallier le déséquilibre entre les deux catégories de y 
#)

#print("Train :", X_train.shape)
#print("Test  :", X_test.shape)
#y_train.value_counts()

# Premier modèle : la régression logistique 

Pour la régression logistique, on découpe notre base en base d'entraînement et de test et on redécoupe la base d'entraînement en base d'entraînement et de validation. Cette dernière sert à tester notre modèle avec plusieurs hyperparamètres distincts 

In [5]:
# Divsion de la base pour la régression logisitque 
X = df.drop('Label', axis=1) 
y = df['Label'] 
y_encoded = y.map({'objective': 1, 'subjective': 0}).astype(float)

print("Correspondance des classes :")
print("1 = objective, 0 = subjective")
print(y_encoded.value_counts())


# Refaire la séparation train/test/val avec les labels encodés
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y_encoded,
    test_size=0.2,
    random_state=3,
    stratify=y_encoded
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=0.25,
    random_state=3,
    stratify=y_temp
)



Correspondance des classes :
1 = objective, 0 = subjective
Label
1.0    635
0.0    365
Name: count, dtype: int64


On cherche à mener la régression logistique avec le meilleur paramètre C (inverse de la régularisation) possible, pour cela on en teste plusieurs sur la base de validation 

In [6]:

# On normalise les données 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val) 
X_test_scaled = scaler.transform(X_test)

# Hyperparamètre à tester
C_VALUES = [0.01, 0.1, 1, 10, 100]

best_C = None
best_val_score = -1.0
best_model = None

for C_val in C_VALUES:
    print(f"Test du modèle avec C = {C_val} ")
    
    # Création et Entraînement
    model = LogisticRegression(random_state=3, max_iter=1000, C=C_val)
    model.fit(X_train_scaled, y_train)
    
    y_pred_train = model.predict(X_train_scaled)
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)

    y_pred_val = model.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_pred_val)
    val_precision = precision_score(y_val, y_pred_val)
    val_recall = recall_score(y_val, y_pred_val)
    val_f1 = f1_score(y_val, y_pred_val)
    
     
    print(f"Train Precision: {train_precision:.4f}")
    print(f"Validation Precision: {val_precision:.4f}")
    print("\n")
    print(f"Train Accuracy: {train_accuracy:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Train Recall: {train_recall:.4f}")
    print(f"Validation Recall: {val_recall:.4f}")
    print(f"Train F1: {train_f1:.4f}")
    print(f"Validation F1-score: {val_f1:.4f}")
    print("\n") 


    if val_precision > best_val_score:
        best_val_score = val_precision
        best_C = C_val
        best_model = model 
        
    

print(f"On sélectionne le paramètre C = {best_C}")
print(f"Meilleur score de validation (précision): {best_val_score:.4f}")




Test du modèle avec C = 0.01 
Train Precision: 0.8259
Validation Precision: 0.8227


Train Accuracy: 0.8267
Validation Accuracy: 0.8200
Train Recall: 0.9213
Validation Recall: 0.9134
Train F1: 0.8710
Validation F1-score: 0.8657


Test du modèle avec C = 0.1 
Train Precision: 0.8517
Validation Precision: 0.8357


Train Accuracy: 0.8550
Validation Accuracy: 0.8350
Train Recall: 0.9344
Validation Recall: 0.9213
Train F1: 0.8911
Validation F1-score: 0.8764


Test du modèle avec C = 1 
Train Precision: 0.8596
Validation Precision: 0.8182


Train Accuracy: 0.8600
Validation Accuracy: 0.8200
Train Recall: 0.9318
Validation Recall: 0.9213
Train F1: 0.8942
Validation F1-score: 0.8667


Test du modèle avec C = 10 
Train Precision: 0.8655
Validation Precision: 0.8125


Train Accuracy: 0.8633
Validation Accuracy: 0.8150
Train Recall: 0.9291
Validation Recall: 0.9213
Train F1: 0.8962
Validation F1-score: 0.8635


Test du modèle avec C = 100 
Train Precision: 0.8698
Validation Precision: 0.8069


Tr

Augmenter C (réduire la pénalisation) réduit les différents scores (accuracy, precision, recall, f1) donc on fait le choix de garder C = 1 pour l'ensemble de test